# [STARTER] Udaplay Project - Part 2

## Part 02 - AI Agent Development

In this part, you'll build an intelligent AI agent that combines local knowledge (RAG) with web search capabilities to answer questions about video games.

The agent will have the following capabilities:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup and Imports

In [17]:
import os
import json
from typing import List, Dict, Any
from dotenv import load_dotenv
import chromadb
from chromadb.utils import embedding_functions
import openai

# Load environment variables
load_dotenv()

# Configure API keys
openai.api_key = os.getenv('OPENAI_API_KEY')

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

### Required Tools Implementation

You need to implement the following tools:
1. `retrieve_game`: Search the vector database for game information
2. `evaluate_retrieval`: Assess the quality of retrieved results
3. `game_web_search`: Perform web searches for additional information

In [18]:
def retrieve_game(query: str) -> List[Dict[str, Any]]:
    """
    Semantic search: Finds most relevant results in the vector DB
    
    Args:
        query: a question about game industry
    
    Returns:
        List of dictionaries, each containing:
        - Platform: like Game Boy, Playstation 5, Xbox 360...
        - Name: Name of the Game
        - YearOfRelease: Year when that game was released for that platform
        - Description: Additional details about the game
        - Genre: Game genre
        - Publisher: Game publisher
    """
    try:
        # Perform semantic search in the collection
        results = collection.query(
            query_texts=[query],
            n_results=5,  # Return top 5 most relevant results
            include=['metadatas', 'documents', 'distances']
        )
        
        # Extract and format the results
        games = []
        if results['metadatas'] and results['metadatas'][0]:
            for metadata in results['metadatas'][0]:
                games.append({
                    'Platform': metadata.get('Platform', 'Unknown'),
                    'Name': metadata.get('Name', 'Unknown'),
                    'YearOfRelease': metadata.get('YearOfRelease', 'Unknown'),
                    'Description': metadata.get('Description', 'No description available'),
                    'Genre': metadata.get('Genre', 'Unknown'),
                    'Publisher': metadata.get('Publisher', 'Unknown')
                })
        
        return games
    except Exception as e:
        print(f"Error in retrieve_game: {e}")
        return []

# Test the retrieve_game tool
print("Testing retrieve_game tool:")
test_results = retrieve_game("racing games")
print(f"Found {len(test_results)} racing games")
if test_results:
    print(json.dumps(test_results[0], indent=2))

Testing retrieve_game tool:
Found 5 racing games
{
  "Platform": "PlayStation 1",
  "Name": "Gran Turismo",
  "YearOfRelease": 1997,
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment"
}


In [19]:
def evaluate_retrieval(query: str, results: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Evaluate the quality of retrieved results
    
    Args:
        query: The original query
        results: List of retrieved game results
    
    Returns:
        Dictionary containing evaluation metrics:
        - relevance_score: 0-1 score of how relevant results are
        - coverage_score: 0-1 score of how well results cover the query
        - confidence: Overall confidence in the results
        - suggestions: Suggestions for improving the search
    """
    try:
        if not results:
            return {
                'relevance_score': 0.0,
                'coverage_score': 0.0,
                'confidence': 0.0,
                'suggestions': 'No results found. Consider broadening your search terms.'
            }
        
        # Simple relevance scoring based on query terms in results
        query_terms = query.lower().split()
        total_relevance = 0
        
        for result in results:
            result_text = f"{result.get('Name', '')} {result.get('Description', '')} {result.get('Genre', '')}".lower()
            matches = sum(1 for term in query_terms if term in result_text)
            total_relevance += matches / len(query_terms) if query_terms else 0
        
        relevance_score = min(1.0, total_relevance / len(results))
        coverage_score = min(1.0, len(results) / 5.0)  # Assuming we want 5 results
        confidence = (relevance_score + coverage_score) / 2
        
        suggestions = []
        if relevance_score < 0.5:
            suggestions.append('Try using more specific game names or genres')
        if coverage_score < 0.8:
            suggestions.append('Consider broadening your search terms')
        
        return {
            'relevance_score': round(relevance_score, 3),
            'coverage_score': round(coverage_score, 3),
            'confidence': round(confidence, 3),
            'suggestions': '; '.join(suggestions) if suggestions else 'Results look good!'
        }
        
    except Exception as e:
        print(f"Error in evaluate_retrieval: {e}")
        return {
            'relevance_score': 0.0,
            'coverage_score': 0.0,
            'confidence': 0.0,
            'suggestions': f'Error occurred: {str(e)}'
        }

# Test the evaluate_retrieval tool
print("Testing evaluate_retrieval tool:")
evaluation = evaluate_retrieval("racing games", test_results)
print(json.dumps(evaluation, indent=2))

Testing evaluate_retrieval tool:
{
  "relevance_score": 0.5,
  "coverage_score": 1.0,
  "confidence": 0.75,
  "suggestions": "Results look good!"
}


In [20]:
def game_web_search(query: str) -> Dict[str, Any]:
    """
    Perform web search for additional game information
    
    Args:
        query: Search query about games
    
    Returns:
        Dictionary containing:
        - search_results: List of search results
        - summary: Summary of findings
        - sources: List of source URLs
    """
    try:
        # Check if Tavily client is available
        if not hasattr(game_web_search, 'tavily_client'):
            try:
                tavily_api_key = os.getenv('TAVILY_API_KEY')
                if tavily_api_key:
                    from tavily import TavilyClient
                    game_web_search.tavily_client = TavilyClient(api_key=tavily_api_key)
                else:
                    game_web_search.tavily_client = None
            except ImportError:
                game_web_search.tavily_client = None
            except Exception:
                game_web_search.tavily_client = None
        
        # If client is not available, return informative message
        if not game_web_search.tavily_client:
            return {
                'search_results': [],
                'summary': 'Tavily API key not found or client not initialized. Web search unavailable.',
                'sources': [],
                'error': 'CLIENT_NOT_AVAILABLE'
            }
        
        # Perform web search using Tavily
        search_response = game_web_search.tavily_client.search(
            query=query,
            search_depth="basic",
            max_results=5
        )
        
        # Extract relevant information
        results = search_response.get('results', [])
        
        search_results = []
        sources = []
        
        for result in results:
            search_results.append({
                'title': result.get('title', 'No title'),
                'content': result.get('content', 'No content'),
                'url': result.get('url', 'No URL')
            })
            sources.append(result.get('url', 'No URL'))
        
        # Create a summary
        summary = f"Found {len(search_results)} relevant results for '{query}'. "
        if search_results:
            summary += "The search covers various aspects of the gaming industry including reviews, news, and historical information."
        
        return {
            'search_results': search_results,
            'summary': summary,
            'sources': sources,
            'error': None
        }
        
    except Exception as e:
        print(f"Error in game_web_search: {e}")
        return {
            'search_results': [],
            'summary': f'Error occurred during web search: {str(e)}',
            'sources': [],
            'error': 'SEARCH_ERROR'
        }

# Test the game_web_search tool
print("Testing game_web_search tool:")
web_results = game_web_search("latest gaming industry trends 2024")
print(json.dumps(web_results, indent=2))

Testing game_web_search tool:
{
  "search_results": [
    {
      "title": "What Are the Biggest Gaming Trends of 2024? | Fast Feed",
      "content": "The biggest social trends in gaming in 2024  In 2024, leading social trends in gaming are esports, online multiplayer and streaming. Professional esports is huge now.",
      "url": "https://blog.frontier.com/2024/04/what-are-the-biggest-gaming-trends-of-2024/"
    },
    {
      "title": "Top 15 Mobile Game Development Trends to Watch Out in 2024-26",
      "content": "Key trends include AR/VR, cloud gaming, blockchain technology, hyper-casual games, and the rise of indie games, among others.",
      "url": "https://syndelltech.com/top-mobile-game-development-trends/"
    },
    {
      "title": "Gaming industry trends to watch in 2025: Distribution ... - GeekWire",
      "content": "As 2024 draws to a close, one advisory firm predicts the video game industry will see a major rebound in 2025 following two years of declines.",
      "ur

### AI Agent Implementation

Now let's create the main AI agent that combines all these tools to answer questions intelligently.

In [21]:
class UdaplayAgent:
    """
    AI Agent for answering questions about video games using RAG and web search
    """
    
    def __init__(self):
        self.conversation_history = []
        self.tools = {
            'retrieve_game': retrieve_game,
            'evaluate_retrieval': evaluate_retrieval,
            'game_web_search': game_web_search
        }
    
    def add_to_history(self, role: str, content: str):
        """Add message to conversation history"""
        self.conversation_history.append({
            'role': role,
            'content': content,
            'timestamp': len(self.conversation_history)
        })
    
    def get_response(self, query: str) -> Dict[str, Any]:
        """
        Main method to get a response from the agent
        
        Args:
            query: User's question about games
        
        Returns:
            Dictionary containing the agent's response and metadata
        """
        try:
            # Add user query to history
            self.add_to_history('user', query)
            
            # Step 1: Try to find information in local knowledge base
            local_results = self.tools['retrieve_game'](query)
            
            # Step 2: Evaluate the quality of local results
            evaluation = self.tools['evaluate_retrieval'](query, local_results)
            
            # Step 3: Decide if web search is needed
            web_results = None
            if evaluation['confidence'] < 0.6 or len(local_results) < 2:
                web_results = self.tools['game_web_search'](query)
            
            # Step 4: Generate comprehensive response
            response = self._generate_response(query, local_results, evaluation, web_results)
            
            # Add agent response to history
            self.add_to_history('assistant', response['answer'])
            
            return response
            
        except Exception as e:
            error_response = {
                'answer': f'Sorry, I encountered an error: {str(e)}',
                'local_results': [],
                'web_results': None,
                'evaluation': {'confidence': 0.0, 'error': str(e)},
                'sources': []
            }
            self.add_to_history('assistant', error_response['answer'])
            return error_response
    
    def _generate_response(self, query: str, local_results: List[Dict], 
                          evaluation: Dict, web_results: Dict = None) -> Dict[str, Any]:
        """Generate a comprehensive response based on available information"""
        
        # Build the response
        answer_parts = []
        sources = []
        
        # Add local knowledge results
        if local_results:
            answer_parts.append("Based on my knowledge base:")
            for i, result in enumerate(local_results[:3], 1):
                answer_parts.append(f"{i}. {result['Name']} ({result['Platform']}, {result['YearOfRelease']}) - {result['Description']}")
            
            # Add evaluation insights
            if evaluation['confidence'] < 0.7:
                answer_parts.append(f"\nNote: My confidence in these results is {evaluation['confidence']}. {evaluation['suggestions']}")
        
        # Add web search results if available
        if web_results and web_results.get('search_results') and not web_results.get('error'):
            answer_parts.append("\nAdditional information from recent sources:")
            for result in web_results['search_results'][:2]:
                answer_parts.append(f"• {result['title']}: {result['content'][:150]}...")
                sources.append(result['url'])
            
            answer_parts.append(f"\n{web_results['summary']}")
        elif web_results and web_results.get('error'):
            answer_parts.append(f"\nNote: {web_results['summary']}")
        
        # If no results found
        if not local_results and not web_results:
            answer_parts.append("I couldn't find specific information about that in my knowledge base or recent sources. ")
            answer_parts.append("Try rephrasing your question or asking about a different aspect of gaming.")
        
        # Combine all parts
        full_answer = "\n\n".join(answer_parts)
        
        return {
            'answer': full_answer,
            'local_results': local_results,
            'web_results': web_results,
            'evaluation': evaluation,
            'sources': sources
        }
    
    def get_conversation_summary(self) -> str:
        """Get a summary of the conversation history"""
        if not self.conversation_history:
            return "No conversation history yet."
        
        summary = f"Conversation Summary ({len(self.conversation_history)} messages):\n"
        for msg in self.conversation_history[-5:]:  # Last 5 messages
            summary += f"{msg['role'].title()}: {msg['content'][:100]}...\n"
        
        return summary

# Initialize the agent
agent = UdaplayAgent()
print("UdaplayAgent initialized successfully!")

UdaplayAgent initialized successfully!


### Testing the Agent

Let's test our agent with various types of questions to see how it performs.

In [22]:
# Test queries to evaluate the agent's performance
test_queries = [
    "What racing games are available?",
    "Tell me about platformer games",
    "What games were released in the 1990s?",
    "Which games are available for PlayStation?"
]

print("Testing the UdaplayAgent with various queries:\n")

for i, query in enumerate(test_queries, 1):
    print(f"\n--- Test Query {i}: {query} ---")
    response = agent.get_response(query)
    
    print(f"Answer: {response['answer'][:300]}...")
    print(f"Confidence: {response['evaluation']['confidence']}")
    print(f"Local Results: {len(response['local_results'])}")
    print(f"Web Results: {'Yes' if response['web_results'] and not response['web_results'].get('error') else 'No'}")
    
    if response['sources']:
        print(f"Sources: {len(response['sources'])} URLs")
    
    print("-" * 50)

Testing the UdaplayAgent with various queries:


--- Test Query 1: What racing games are available? ---
Answer: Based on my knowledge base:

1. Gran Turismo 5 (PlayStation 3, 2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.

2. Gran Turismo (PlayStation 1, 1997) - A realistic racing simulator featuring a wide array of cars and tracks, ...
Confidence: 0.6
Local Results: 5
Web Results: No
--------------------------------------------------

--- Test Query 2: Tell me about platformer games ---
Answer: Based on my knowledge base:

1. Super Mario 64 (Nintendo 64, 1996) - A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.

2. Super Mario World (Super Nintendo Entertainment System (SNES), 1990) - A classic platformer where Mario emba...
Confidence: 0.66
Local Results: 5
Web Results: No
--------------------------------------------------

--- Test Query 3

### Interactive Conversation

Now you can have an interactive conversation with the agent. Ask it questions about games!

In [23]:
def chat_with_agent():
    """Interactive chat function with the agent"""
    print("Welcome to Udaplay! Ask me anything about video games.")
    print("Type 'quit' to exit, 'history' to see conversation history, or 'summary' for a summary.\n")
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            print("\nThanks for chatting! Goodbye!")
            break
        elif user_input.lower() == 'history':
            print("\nConversation History:")
            for msg in agent.conversation_history:
                print(f"{msg['role'].title()}: {msg['content'][:100]}...")
            continue
        elif user_input.lower() == 'summary':
            print("\n" + agent.get_conversation_summary())
            continue
        elif not user_input:
            continue
        
        # Get response from agent
        response = agent.get_response(user_input)
        
        print(f"\nAgent: {response['answer']}")
        
        # Show confidence and sources if available
        if response['evaluation']['confidence'] < 0.7:
            print(f"\n[Confidence: {response['evaluation']['confidence']}]")
        if response['sources']:
            print(f"\nSources: {', '.join(response['sources'][:2])}")

# Uncomment the line below to start interactive chat
# chat_with_agent()

# For now, let's test with a specific question
print("Testing with a specific question:")
question = "What are some popular racing games from the 1990s?"
response = agent.get_response(question)
print(f"\nQuestion: {question}")
print(f"\nAnswer: {response['answer']}")

Testing with a specific question:

Question: What are some popular racing games from the 1990s?

Answer: Based on my knowledge base:

1. Gran Turismo (PlayStation 1, 1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.

2. Gran Turismo 5 (PlayStation 3, 2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.

3. Super Mario World (Super Nintendo Entertainment System (SNES), 1990) - A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser.


Note: My confidence in these results is 0.567. Try using more specific game names or genres


Additional information from recent sources:

• What are the Best Racing Games From the 90s?: What are the Best Racing Games From the 90s? · 1 Mario Kart 64 (1996) · 2 Gran Turismo (1997) · 3 Ridge Racer Type 4 (1999) · 4 Sega Rally...

• The 11 best racing games of the 1990s 

### Advanced Features and Enhancements

Here are some ideas for enhancing your agent:

1. **Long-term Memory**: Store conversation history in a database
2. **Better Evaluation**: Use LLM-based evaluation for more accurate assessment
3. **Tool Chaining**: Automatically chain tools based on query complexity
4. **Response Templates**: Create structured response templates for different query types
5. **Error Handling**: Implement more sophisticated error handling and recovery
6. **Performance Metrics**: Track and analyze agent performance over time

In [24]:
# Example of how you could enhance the agent with better evaluation
def enhanced_evaluation(query: str, results: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Enhanced evaluation using LLM for better assessment
    This is an example of how you could improve the evaluation
    """
    try:
        if not results:
            return {
                'relevance_score': 0.0,
                'coverage_score': 0.0,
                'confidence': 0.0,
                'suggestions': 'No results found. Consider broadening your search terms.'
            }
        
        # Create a prompt for the LLM to evaluate results
        evaluation_prompt = f"""
        Evaluate the relevance of these game results to the query: "{query}"
        
        Results:
        {json.dumps(results, indent=2)}
        
        Rate each result from 0-10 for relevance and provide an overall confidence score.
        """
        
        # This would use OpenAI's API for evaluation
        # For now, return the basic evaluation
        return evaluate_retrieval(query, results)
        
    except Exception as e:
        print(f"Error in enhanced_evaluation: {e}")
        return evaluate_retrieval(query, results)

print("Enhanced evaluation function created (example implementation)")

Enhanced evaluation function created (example implementation)


## Conclusion

Congratulations! You've successfully implemented:

✅ **Part 1**: Offline RAG with ChromaDB
✅ **Part 2**: AI Agent with multiple tools
✅ **Required Tools**: retrieve_game, evaluate_retrieval, game_web_search
✅ **Agent Capabilities**: Local knowledge + Web search + Conversation state

Your Udaplay agent can now:
- Answer questions using local game knowledge
- Search the web for additional information
- Maintain conversation context
- Provide structured responses with confidence scores
- Suggest improvements when results are insufficient

### Next Steps:
1. Test with various game-related queries
2. Enhance the evaluation system
3. Add more sophisticated memory management
4. Implement additional tools as needed
5. Deploy and monitor performance

Happy gaming research! 🎮